In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split


import tensorflow as tf
from keras.models import Sequential,load_model
from keras import layers,Input
from keras.losses import binary_crossentropy
from keras.layers import Dense
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import keras.backend as K

2023-11-13 10:29:56.968597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 10:29:58.885693: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
Q0state0=np.load('../Data/new/2gsps/Q0state0_her.npy')
Q0state1=np.load('../Data/new/2gsps/Q0state1_her.npy')
Q0output0=np.zeros((Q0state0.shape[0]))
Q0output1=np.ones((Q0state1.shape[0]))
print(Q0state0.shape, Q0state1.shape)

Q1state0=np.load('../Data/new/2gsps/Q1state0_her.npy')
Q1state1=np.load('../Data/new/2gsps/Q1state1_her.npy')
indices = np.arange(Q1state1.shape[0])
np.random.shuffle(indices)
selected_indices = indices[:950]
Q1state0 = Q1state0[selected_indices]
Q1state1 = Q1state1[selected_indices]
Q1output0=np.zeros((Q1state0.shape[0]))
Q1output1=np.ones((Q1state1.shape[0]))
print(Q1state0.shape, Q1state1.shape)

(967, 8192, 2) (948, 8192, 2)
(950, 8192, 2) (950, 8192, 2)


In [4]:
st,ed=700,3700
Q0state0=np.mean(Q0state0[:,st:ed,:], axis=1)
Q0state1=np.mean(Q0state1[:,st:ed,:], axis=1)
Q1state0=np.mean(Q1state0[:,st:ed,:], axis=1)
Q1state1=np.mean(Q1state1[:,st:ed,:], axis=1)
print(Q0state0.shape, Q0state1.shape, Q1state0.shape, Q1state1.shape)

(967, 2) (948, 2) (950, 2) (950, 2)


In [5]:
print(Q0state0,Q1state0)
Q0state0=np.hstack((Q0state0,np.zeros((Q0state0.shape[0],1))))
Q0state1=np.hstack((Q0state1,np.zeros((Q0state1.shape[0],1))))
Q1state0=np.hstack((Q1state0,np.ones((Q1state0.shape[0],1))))
Q1state1=np.hstack((Q1state1,np.ones((Q1state1.shape[0],1))))
print(Q0state0,Q1state0)
print(Q0state0.shape, Q0state1.shape, Q1state0.shape, Q1state1.shape)

[[-1.0756128  -5.86365771]
 [ 2.7664248  -5.22110215]
 [-1.60407841 -3.22271376]
 ...
 [-0.88298124 -4.83261184]
 [ 4.55678431 -2.54877623]
 [ 1.75797679 -4.80339969]] [[-0.79316789 -5.33455145]
 [-2.61060736  0.40732224]
 [-2.31076677 -2.93657597]
 ...
 [-4.09046704 -2.12936527]
 [-2.96161866 -3.3928315 ]
 [-1.54820235 -3.89825074]]
[[-1.0756128  -5.86365771  0.        ]
 [ 2.7664248  -5.22110215  0.        ]
 [-1.60407841 -3.22271376  0.        ]
 ...
 [-0.88298124 -4.83261184  0.        ]
 [ 4.55678431 -2.54877623  0.        ]
 [ 1.75797679 -4.80339969  0.        ]] [[-0.79316789 -5.33455145  1.        ]
 [-2.61060736  0.40732224  1.        ]
 [-2.31076677 -2.93657597  1.        ]
 ...
 [-4.09046704 -2.12936527  1.        ]
 [-2.96161866 -3.3928315   1.        ]
 [-1.54820235 -3.89825074  1.        ]]
(967, 3) (948, 3) (950, 3) (950, 3)


In [6]:
x=np.vstack((Q0state0,Q0state1,Q1state0,Q1state1))
print(x.shape)
y=np.hstack((Q0output0,Q0output1,Q1output0,Q1output1))
print(y.shape)

(3815, 3)
(3815,)


In [7]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=45)
print(x_train.shape)
print(y_train.sum())

(3052, 3)
1523.0


In [66]:
model=SVC(decision_function_shape='ovo')
model.fit(x_train,y_train)
print(model.score(x_test,y_test))
pred=model.predict(x_test)

0.9724770642201835


In [67]:
def arch():
    model=Sequential()
    model.add(Input(shape=(3), name='Input Layer'))
    model.add(Dense(8,activation='relu', name='HiddenLayer1'))
    model.add(Dense(4,activation='relu', name='HiddenLayer2'))
    model.add(Dense(1,activation='sigmoid', name='OuputLayer'))
    return model
arch().summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 HiddenLayer1 (Dense)        (None, 8)                 32        
                                                                 
 HiddenLayer2 (Dense)        (None, 4)                 36        
                                                                 
 OuputLayer (Dense)          (None, 1)                 5         
                                                                 
Total params: 73 (292.00 Byte)
Trainable params: 73 (292.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [68]:
def bceloss(y_true, y_pred):
    # Clip the prediction value to prevent log(0) error
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Calculate the weighted binary cross entropy loss
    loss = -(0.1 * (y_true * K.log(y_pred)) + 0.9*((1 - y_true) * K.log(1 - y_pred)))
    return K.mean(loss, axis=-1)

model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN2/tmp/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [69]:
H=model.fit(x_train,y_train,
          validation_split=0.2,
          epochs=100,batch_size=64,
          callbacks=callbacks_list,
          verbose=1)

Epoch 1/100
 1/39 [..............................] - ETA: 17s - loss: 1.9748 - accuracy: 0.3125
Epoch 1: val_accuracy improved from -inf to 0.41408, saving model to ../Model/NN2/tmp/01-0.4141.h5
39/39 [==============================] - 1s 6ms/step - loss: 1.6436 - accuracy: 0.3519 - val_loss: 1.3666 - val_accuracy: 0.4141
Epoch 2/100
 1/39 [..............................] - ETA: 0s - loss: 1.2662 - accuracy: 0.5469
Epoch 2: val_accuracy improved from 0.41408 to 0.41735, saving model to ../Model/NN2/tmp/02-0.4173.h5
39/39 [==============================] - 0s 4ms/step - loss: 1.2498 - accuracy: 0.3953 - val_loss: 1.0653 - val_accuracy: 0.4173
Epoch 3/100
 1/39 [..............................] - ETA: 0s - loss: 1.1135 - accuracy: 0.4062

/global/homes/n/nrvora/.conda/envs/readout/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 3: val_accuracy improved from 0.41735 to 0.42226, saving model to ../Model/NN2/tmp/03-0.4223.h5
39/39 [==============================] - 0s 4ms/step - loss: 1.0058 - accuracy: 0.3994 - val_loss: 0.8948 - val_accuracy: 0.4223
Epoch 4/100
 1/39 [..............................] - ETA: 0s - loss: 0.9272 - accuracy: 0.4688
Epoch 4: val_accuracy did not improve from 0.42226
39/39 [==============================] - 0s 3ms/step - loss: 0.8731 - accuracy: 0.4002 - val_loss: 0.8001 - val_accuracy: 0.4059
Epoch 5/100
 1/39 [..............................] - ETA: 0s - loss: 0.7686 - accuracy: 0.5312
Epoch 5: val_accuracy did not improve from 0.42226
39/39 [==============================] - 0s 3ms/step - loss: 0.7903 - accuracy: 0.3789 - val_loss: 0.7288 - val_accuracy: 0.3797
Epoch 6/100
 1/39 [..............................] - ETA: 0s - loss: 0.7614 - accuracy: 0.3438
Epoch 6: val_accuracy did not improve from 0.42226
39/39 [==============================] - 0s 4ms/step - loss: 0.7231 - ac

In [8]:
model=load_model('../Model/NN2/tmp/64-0.9771.h5', compile=False)
count=0
count1=0
y_pred=model.predict(x_test)
for i in range(y_pred.shape[0]):
    if y_test[i]==0:
        count+=1
        if y_pred[i]<=0.5:
            count1+=1
print(count,count1)

2023-11-13 10:30:19.232060: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


24/24 [==============================] - 0s 649us/step
388 382
